数据预处理（请自行安装所需包）

In [2]:
%pip install scikit-learn
%pip install pandas
%pip install joblib

You should consider upgrading via the 'd:\python3.10.1\python.exe -m pip install --upgrade pip' command.



Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\python3.10.1\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'd:\python3.10.1\python.exe -m pip install --upgrade pip' command.


In [3]:

from load_data import *

#考虑到时间，取前100k条
X,Y=readcsv('data/Amazon_Unlocked_Mobile.csv',100000)
X_train=X[0:70000]

X_test=X[70000:100000]

Y_train=Y[0:70000]
Y_test=Y[70000:100000]

d:\python3.10.1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NB训练与评价

In [13]:
import os
import joblib

# TFIDF模型
from sklearn.feature_extraction.text import TfidfVectorizer
# 管道模型可将两个算法进行连接
from sklearn.pipeline import Pipeline

In [10]:
# 朴素贝叶斯
from sklearn.naive_bayes import MultinomialNB
# 将TFIDF模型和朴素贝叶斯算法连接
TFIDF_NB_Sentiment_Model = Pipeline([
    ('TFIDF', TfidfVectorizer()),
    ('NB', MultinomialNB())
])

nbm = TFIDF_NB_Sentiment_Model.fit(X_train,Y_train)
nb_train_score = TFIDF_NB_Sentiment_Model.score(X_test,Y_test)
os.makedirs('model/nb',exist_ok=True)
joblib.dump(TFIDF_NB_Sentiment_Model, 'model/nb/tfidf_nb.model')
print(nb_train_score)
 
 


0.6557333333333333


SVM

In [19]:
 
#SVM进行训练

from sklearn.svm import SVC

TFIDF_SVM_Sentiment_Model = Pipeline([
    ('TFIDF', TfidfVectorizer()),
    ('SVM', SVC(C=0.5,cache_size=7000,max_iter=300,verbose=True))
])
TFIDF_SVM_Sentiment_Model.fit(X_train,Y_train)
svm_test_score = TFIDF_SVM_Sentiment_Model.score(X_test,Y_test)
os.makedirs('model/svm',exist_ok=True)
joblib.dump(TFIDF_SVM_Sentiment_Model, 'model/svm/tfidf_svm.model')
print(svm_test_score)

[LibSVM]

d:\python3.10.1\lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


0.5381333333333334


logistic regression

In [20]:
from sklearn.linear_model import LogisticRegression
 
TFIDF_LR_Sentiment_Model = Pipeline([
    ('TFIDF', TfidfVectorizer()),
    ('logisticregression', LogisticRegression(max_iter=300,multi_class='multinomial'))
])
TFIDF_LR_Sentiment_Model.fit(X_train,Y_train)
lr_test_score = TFIDF_LR_Sentiment_Model.score(X_test,Y_test)
os.makedirs('model/lr',exist_ok=True)
joblib.dump(TFIDF_LR_Sentiment_Model, 'model/lr/tfidf_lr.model')
print(lr_test_score)

d:\python3.10.1\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7314333333333334


SVD+logistic regression

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
TFIDF_SVD_LR_Sentiment_Model = Pipeline([
    ('TFIDF', TfidfVectorizer()),
    ('SVD',TruncatedSVD(1024)),
    ('logisticregression', LogisticRegression(max_iter=300,multi_class='multinomial'))
])
TFIDF_SVD_LR_Sentiment_Model.fit(X_train,Y_train)
lr_test_score = TFIDF_SVD_LR_Sentiment_Model.score(X_test,Y_test)
os.makedirs('model/lr',exist_ok=True)
joblib.dump(TFIDF_SVD_LR_Sentiment_Model, 'model/lr/tfidf_svd_lr.model')
print(lr_test_score)

0.7105333333333334


预测函数API

In [22]:
def Predict(text):
    #模型路径自行更改
    model = joblib.load('model/lr/tfidf_svd_lr.model')

    y_pre =model.predict([text])
    print(f"The predicted rating is {y_pre[0]}/5.")

Predict("I like it but there are some shortcomings")

The predicted rating is 4/5.
